In [1]:
# Install dependencies
!pip install timm -q

import kagglehub
import os, torch
import matplotlib.pyplot as plt
from PIL import Image

# Download dataset
dataset_path = kagglehub.dataset_download("salviohexia/isic-2019-skin-lesion-images-for-classification")
print("Dataset path:", dataset_path)
print("Contents:", os.listdir(dataset_path))

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

100%|██████████| 9.10G/9.10G [05:08<00:00, 31.7MB/s]

Extracting files...


Dataset path: /root/.cache/kagglehub/datasets/salviohexia/isic-2019-skin-lesion-images-for-classification/versions/1
Contents: ['ISIC_2019_Training_Metadata.csv', 'BKL', 'AK', 'ISIC_2019_Training_GroundTruth.csv', 'MEL', 'NV', 'VASC', 'BCC', 'SCC', 'DF']
Device: cuda


In [2]:
import pandas as pd

# Check classes
CLASS_NAMES = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']
for cls in CLASS_NAMES:
    path = os.path.join(dataset_path, cls)
    count = len(os.listdir(path)) if os.path.exists(path) else 0
    print(f"{cls}: {count} images")

# Check metadata
csv_path = os.path.join(dataset_path, "ISIC_2019_Training_Metadata.csv")
df = pd.read_csv(csv_path)
print("\nMetadata shape:", df.shape)
print(df.head())

AK: 867 images
BCC: 3323 images
BKL: 2624 images
DF: 239 images
MEL: 4522 images
NV: 12875 images
SCC: 628 images
VASC: 253 images

Metadata shape: (25331, 5)
          image  age_approx anatom_site_general lesion_id     sex
0  ISIC_0000000        55.0      anterior torso       NaN  female
1  ISIC_0000001        30.0      anterior torso       NaN  female
2  ISIC_0000002        60.0     upper extremity       NaN  female
3  ISIC_0000003        30.0     upper extremity       NaN    male
4  ISIC_0000004        80.0     posterior torso       NaN    male


In [3]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, random_split
import torchvision.transforms as T
from PIL import Image
from collections import Counter

# ── Metadata Preprocessing ──────────────────────────────────────
def preprocess_metadata(df):
    df = df.copy()

    # Rename image column to match filenames
    df = df.rename(columns={'image': 'isic_id'})

    # Age — fill missing with median, normalize
    df['age_approx'] = df['age_approx'].fillna(df['age_approx'].median())
    df['age_approx'] = (df['age_approx'] - df['age_approx'].mean()) / df['age_approx'].std()

    # Sex — one hot
    df['sex'] = df['sex'].fillna('unknown')
    df['sex_male']    = (df['sex'] == 'male').astype(float)
    df['sex_female']  = (df['sex'] == 'female').astype(float)
    df['sex_unknown'] = (df['sex'] == 'unknown').astype(float)

    # Anatomical site — one hot
    sites = ['anterior torso', 'posterior torso', 'upper extremity',
             'lower extremity', 'head/neck', 'palms/soles', 'oral/genital']
    df['anatom_site_general'] = df['anatom_site_general'].fillna('unknown')
    for site in sites:
        col = 'site_' + site.replace('/', '_').replace(' ', '_')
        df[col] = (df['anatom_site_general'] == site).astype(float)

    meta_cols = ['age_approx', 'sex_male', 'sex_female', 'sex_unknown'] + \
                ['site_' + s.replace('/', '_').replace(' ', '_') for s in sites]

    return df, meta_cols

df, meta_cols = preprocess_metadata(df)
print(f"✅ Metadata ready | Features: {len(meta_cols)} | Columns: {meta_cols}")

# ── Dataset ──────────────────────────────────────────────────────
CLASS_NAMES = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']
CLASS_TO_IDX = {c: i for i, c in enumerate(CLASS_NAMES)}

class ISICDataset(Dataset):
    def __init__(self, dataset_path, df, meta_cols, transform=None):
        self.meta_df   = df.set_index('isic_id')
        self.meta_cols = meta_cols
        self.transform = transform
        self.samples   = []

        for cls in CLASS_NAMES:
            cls_path = os.path.join(dataset_path, cls)
            if not os.path.exists(cls_path):
                continue
            for img_file in os.listdir(cls_path):
                if img_file.lower().endswith('.jpg'):
                    img_id = img_file.replace('.jpg', '')
                    self.samples.append({
                        'path':  os.path.join(cls_path, img_file),
                        'label': CLASS_TO_IDX[cls],
                        'id':    img_id
                    })

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s   = self.samples[idx]
        img = Image.open(s['path']).convert('RGB')
        if self.transform:
            img = self.transform(img)

        if s['id'] in self.meta_df.index:
            meta = torch.tensor(
                self.meta_df.loc[s['id'], self.meta_cols].values.astype(np.float32))
        else:
            meta = torch.zeros(len(self.meta_cols), dtype=torch.float32)

        return img, meta, s['label']

# ── Transforms ───────────────────────────────────────────────────
train_transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    T.RandomRotation(20),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# ── Load & Split ─────────────────────────────────────────────────
full_ds    = ISICDataset(dataset_path, df, meta_cols, transform=train_transform)
train_size = int(0.8 * len(full_ds))
val_size   = len(full_ds) - train_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])
val_ds.dataset.transform = val_transform

# ── Weighted Sampler (fix class imbalance) ───────────────────────
labels       = [full_ds.samples[i]['label'] for i in range(len(full_ds))]
counts       = Counter(labels)
class_weights = torch.tensor([len(labels) / (len(CLASS_NAMES) * counts[i])
                               for i in range(len(CLASS_NAMES))], dtype=torch.float32)
sample_weights = [class_weights[l] for l in labels]
train_indices  = train_ds.indices
train_weights  = [sample_weights[i] for i in train_indices]
sampler        = WeightedRandomSampler(train_weights, len(train_weights))

# ── DataLoaders ──────────────────────────────────────────────────
train_loader = DataLoader(train_ds, batch_size=32, sampler=sampler,
                          num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False,
                          num_workers=2, pin_memory=True)

print(f"✅ Train: {len(train_ds)} | Val: {len(val_ds)}")
print(f"✅ Class weights: {dict(zip(CLASS_NAMES, class_weights.numpy().round(2)))}")

✅ Metadata ready | Features: 11 | Columns: ['age_approx', 'sex_male', 'sex_female', 'sex_unknown', 'site_anterior_torso', 'site_posterior_torso', 'site_upper_extremity', 'site_lower_extremity', 'site_head_neck', 'site_palms_soles', 'site_oral_genital']
✅ Train: 20264 | Val: 5067
✅ Class weights: {'AK': np.float32(3.65), 'BCC': np.float32(0.95), 'BKL': np.float32(1.21), 'DF': np.float32(13.25), 'MEL': np.float32(0.7), 'NV': np.float32(0.25), 'SCC': np.float32(5.04), 'VASC': np.float32(12.52)}


In [4]:
import torch
print(torch.cuda.is_available())       # Must be True
print(torch.cuda.get_device_name(0))   # Must show T4
device = torch.device("cuda")
print(device)                          # cuda


True
Tesla T4
cuda


In [5]:
import timm
import torch.nn as nn

# ── Model ─────────────────────────────────────────────────────────
class SkinCancerModel(nn.Module):
    def __init__(self, num_classes=8, metadata_dim=11):
        super().__init__()

        # Image branch — EfficientNet-B2
        self.image_branch = timm.create_model('efficientnet_b2', pretrained=True)
        image_out = self.image_branch.classifier.in_features  # 1408
        self.image_branch.classifier = nn.Identity()

        self.image_fc = nn.Sequential(
            nn.Linear(image_out, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Metadata branch
        self.meta_fc = nn.Sequential(
            nn.Linear(metadata_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        # Fusion
        self.fusion = nn.Sequential(
            nn.Linear(512 + 128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, image, metadata):
        img_feat  = self.image_branch(image)
        img_feat  = self.image_fc(img_feat)
        meta_feat = self.meta_fc(metadata)
        fused     = torch.cat([img_feat, meta_feat], dim=1)
        return self.fusion(fused)

# ── Init ──────────────────────────────────────────────────────────
model = SkinCancerModel(num_classes=8, metadata_dim=11).to(device)
print(f"✅ Model ready on {device}")

# Quick sanity check
dummy_img  = torch.randn(2, 3, 224, 224).to(device)
dummy_meta = torch.randn(2, 11).to(device)
out        = model(dummy_img, dummy_meta)
print(f"✅ Output shape: {out.shape}")  # should be torch.Size([2, 8])

# Count parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ Trainable parameters: {total_params:,}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

✅ Model ready on cuda
✅ Output shape: torch.Size([2, 8])
✅ Trainable parameters: 8,599,434


In [ ]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR

# ── Loss & Optimizer ──────────────────────────────────────────────
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

optimizer = AdamW([
    {'params': model.image_branch.parameters(), 'lr': 1e-4},
    {'params': model.image_fc.parameters(),     'lr': 3e-4},
    {'params': model.meta_fc.parameters(),      'lr': 3e-4},
    {'params': model.fusion.parameters(),       'lr': 3e-4},
], weight_decay=1e-4)

EPOCHS    = 15
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

# ── Training Loop ─────────────────────────────────────────────────
best_val_acc = 0

for epoch in range(EPOCHS):
    # Train
    model.train()
    train_loss, correct, total = 0, 0, 0

    for imgs, metas, labels in train_loader:
        imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs, metas)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct    += (outputs.argmax(1) == labels).sum().item()
        total      += labels.size(0)

    # Validate
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for imgs, metas, labels in val_loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs     = model(imgs, metas)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total   += labels.size(0)

    val_acc = val_correct / val_total
    scheduler.step()

    print(f"Epoch {epoch+1:02d}/{EPOCHS} | "
          f"Loss: {train_loss/len(train_loader):.4f} | "
          f"Train Acc: {correct/total:.4f} | "
          f"Val Acc: {val_acc:.4f}"
          + (" ✅ saved" if val_acc > best_val_acc else ""))

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

print(f"\n🏆 Best Val Acc: {best_val_acc:.4f}")